In [1]:
## install the pytorch3d package; hit 'shift+enter' keys to run
!conda create --name pytorch3d --yes
!conda activate pytorch3d
!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch --yes
!conda install -c conda-forge -c takatosp1 fvcore --yes
!conda install pytorch3d -c pytorch3d --yes
!python3 -m pip install PyGEL3D tqdm pandas scipy opencv-python 
!python3 -m pip install --upgrade matplotlib

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda/envs/pytorch3d


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate pytorch3d
#
# To deactivate an active environment, use
#
#     $ conda deactivate


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
If your shell is Bash or a Bourne variant, enable conda for the current user with

    $ echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc

or, for all users, enable conda with

    $ sudo ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh

The options above will permanently enable the 'conda' command, but they do NOT
put conda's base (root) environment on PATH.  To do so, ru

In [14]:
## download the concerthall optimization code base; hit 'shift+enter' keys to run
!git clone https://github.com/alexacarlson/concerthall.git /storage/concerthall

Cloning into '/storage/concerthall'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (597/597), done.
remote: Compressing objects: 100% (560/560), done.
remote: Total 597 (delta 121), reused 447 (delta 32), pack-reused 0
Receiving objects: 100% (597/597), 21.33 MiB | 26.16 MiB/s, done.
Resolving deltas: 100% (121/121), done.
Checking connectivity... done.
Checking out files: 100% (390/390), done.


In [7]:
## Train the graph convolutional neural network; hit 'shift+enter' keys to run
## NOTES on Training the Graph Convolutional Neural network to map 3D meshes --> audio quality parameters:
## To train your own graph convolutional network, you first need to create a folder in paperspace storage 
## at `/storage/concert_dataset` upload your data there. All of your meshes should be `.obj` files and
## should exist in a subdirectory of the dataset: `/storage/concert_dataset/OBJdatabase/`. 
## You will need to have a csv file of acoustic paramters (where each line is in the format `mesh.obj acoust_param1 acoust_param2 ... acoust_param10`) 
## located at and named `/storage/concert_dataset/AcousticParameters.csv`.
## The results (network weights and training loss values) will be saved to `/storage/mesh2acoustic_training_results`.
!cd /storage/concerthall && bash run_train_graphconv_regression.sh

RANDOM_SEED: 0
DATA:
  PATH: /storage/concert_dataset
BASE_MODEL:
  CLASSES: 10
  CONV_INIT: normal
  HIDDEN_DIMS: [128, 256, 512]
  INPUT_MESH_FEATS: 3
OPTIM:
  BATCH_SIZE: 4
  CLIP_GRADIENTS: 12.5
  EPOCH: 10
  LR: 0.0002
  MOMENTUM: 0.9
  VAL_BATCH_SIZE: 4
  WEIGHT_DECAY: 0.005
  WORKERS: 8
CHECKPOINT:
  best_acc: 0.0
  best_loss: 1.7976931348623157e+308
  experiment_path: /storage/mesh2acoustic_training_results/exp_03_23_19_41_34_concerthalloptim
  full_experiment_name: exp_03_23_19_41_34_concerthalloptim

config_yml          : config/mesh2audioparams_train.yml
config_override     : []
Splitting Dataset..
Training Samples: 24
Validation Samples: 3

 ***************** Training *****************
  0%|                                                    | 0/10 [00:00<?, ?it/s]Epoch: 0

	Training Loss: 35640240.166666664
 10%|████▍                                       | 1/10 [00:01<00:15,  1.72s/it]Epoch: 1

	Training Loss: 30029596.75
 20%|████████▊                                   |

In [16]:
## run the optimization framework; hit 'shift+enter' keys to run
## IMPORTANT INFO about running the acoustic optimization script:
## The Command Format for this script: 
## bash run_acousticoptim.sh INPUT_MESH SILHOUETTE_DEFORM_FLAG SILHOUETTE_REF_IMG ACOUSTIC_DEFORM_FLAG WHICH_ACOUSTIC_PARAMS OUTPUT_NAME
## where `INPUT_MESH_FILEPATH` is the absolute filepath of the mesh you would like to deform, 
##       `TRAINED_GRAPH_FILEPATH` is the filepath to the trained mesh2acoustic graph convolutional neural network,
##       `SILHOUETTE_DEFORM_FLAG` is either True or False depending on if you would like to deform the mesh to optimize for a specific 2D silhouette, 
##       `SILHOUETTE_REF_IMG_FILEPATH` is the absolute filepath of the silhouette image you would like to use for mesh deformation, 
##       `CAMPOSITION_FILEPATH` is the absolute filepath of the text file that contains camera positions to render silhouette from mesh. 
##                           Note that each line of the file describes the location of one camera (e.g., two lines will yeild two cameras). 
##                           The location of each camera is described by its distance from the mesh, the elevation of the camera, and the angle from the x axis. 
##       `ACOUSTIC_DEFORM_FLAG` is either True or False depending on if you would like to deform the mesh to optimize for acostic quality, 
##       `WHICH_ACOUSTIC_PARAMS` is a string of the 10 acoustic quality parameters (each parameter needs to be separated by a comma with no spaces in between, see example below), 
##       `OUTPUT_FILENAME` is the absolute filepath where you would like the deformed output mesh to be saved.
##
## Example Command:
#!cd /storage/concerthall && bash run_acousticoptim.sh \
#                            INPUT_MESH_FILEPATH \
#                            TRAINED_GRAPH_FILEPATH \
#                            SILHOUETTE_DEFORM_FLAG \
#                            SILHOUETTE_REF_IMG_FILEPATH \
#                            CAMPOSITION_FILEPATH \
#                            ACOUSTIC_DEFORM_FLAG \
#                            WHICH_ACOUSTIC_PARAMS \
#                            OUTPUT_FILENAME
#
!cd /storage/concerthall && bash run_acousticoptim.sh \
                            "/storage/3Dmodels/bench.obj" \
                            "/storage/concerthall_model_epoch99.pkl"\
                            "True" \
                            "/storage/2Dmodels/new000343.png" \
                            "/storage/camppos.txt" \
                            "True" \
                            '2660,24920,1.25,2066,17.7,57.7,0.54,0.31,9.37,1.55'  \
                            "test_sil_deform.obj"


Saving optim results to /storage/results_acousticoptim_2660,24920,1.25,2066,17.7,57.7,0.54,0.31,9.37,1.55_multicamVertoptim_new000343

 ***************** Deforming *****************
Iteration: 0 Loss: 604594300.0
Iteration: 50 Loss: 595689000.0
Iteration: 100 Loss: 578892500.0
Iteration: 150 Loss: 543398600.0
Iteration: 200 Loss: 476858370.0
Iteration: 250 Loss: 378621600.0
Iteration: 300 Loss: 278684640.0
Iteration: 350 Loss: 202852240.0
Iteration: 400 Loss: 145377460.0
Iteration: 450 Loss: 101695304.0
Iteration: 500 Loss: 68703336.0
